# Imports and Constants

An API key must be given. Registration required first.

In [4]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%autoreload 2
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil import parser

from dotenv import load_dotenv

from tqdm import tqdm, tqdm_notebook

# import metalhistory as mh
import metalhistory.data_query_functions as dqf

In [6]:
tqdm.pandas()

# For devs: Overview of functions
These functions from utils.py work (untested) so far:

In [7]:
lastfm = dqf.LastFM()
lastfm.get_album_info('Death', 'Symbolic', verbose=1)
# lastfm.get_album_matches('Burzum', 'Filosofem',  verbose=1)
# lastfm.get_track_info('Black Sabbath', 'Paranoid', 'War Pigs', verbose=1)

Generated API Request: http://ws.audioscrobbler.com/2.0/?&api_key=2e171c2f29c3f9b0258954e2edb289c3&method=album.getinfo&artist=Death&album=Symbolic&format=json


{'name': 'Symbolic',
 'artist': 'Death',
 'mbid': '321a3c33-9310-4b9f-b104-762e465ec60f',
 'url': 'https://www.last.fm/music/Death/Symbolic',
 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/459b51d39e5447e8c7f86ea0a8b34487.png',
   'size': 'small'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/459b51d39e5447e8c7f86ea0a8b34487.png',
   'size': 'medium'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/459b51d39e5447e8c7f86ea0a8b34487.png',
   'size': 'large'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/459b51d39e5447e8c7f86ea0a8b34487.png',
   'size': 'extralarge'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/459b51d39e5447e8c7f86ea0a8b34487.png',
   'size': 'mega'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/459b51d39e5447e8c7f86ea0a8b34487.png',
   'size': ''}],
 'listeners': '180443',
 'playcount': '5710284',
 'tracks': {'track': [{'name': 'Symbolic',
    'url': 'https://www.last.fm/music/Death/_/Symboli

# Data Preprocessing

In [39]:
df_csv = pd.read_csv('data/MA_10k_albums.csv')
# df_csv = df_csv.dropna(axis=0)
df_csv

,artist,album,MA_score
0,Slayer,Reign in Blood,36.01
1,Metallica,Kill 'Em All,33.39
2,Hades Archer,Penis Metal,32.67
3,Iron Maiden,Iron Maiden,32.38
4,Metallica,Master of Puppets,31.83
...,...,...,...
9995,Iron Maiden,Live at the Rainbow,1.92
9996,Jorn,Worldchanger,1.92
9997,Juggernaut,Trouble Within,1.92
9998,Lacrimas Profundere,Memorandum,1.92


## Collected all instances of an album

In [40]:
lastfm = dqf.LastFM()
results_df = pd.DataFrame()

LIMIT=100

df_head = df_csv.head(LIMIT)

for idx, row in tqdm_notebook(df_head.iterrows(), total=df_head.shape[0], desc='artists'):
    artist = row['artist']
    album = row['album']
    
    print('Querying for', artist, '-', album)
    matches = list(lastfm.get_album_matches(artist, album)['results']['albummatches'].values())[0]
    # print(matches)

    nr_matches =  len(matches)
    nr_kept_matches = 0

    for match in tqdm_notebook(matches, desc='album matches', leave=False):
        if match['artist'] == artist:
            results_df = results_df.append({
                'artist': artist,
                'album': album,
                'album_instance': match['name'],
                'lastfm_info': lastfm.get_album_info(match['artist'], match['name'])
            }, ignore_index=True)
            nr_kept_matches += 1        
    print('Kept', nr_kept_matches, 'out of', nr_matches, 'matches for', artist, '-', album, end='\n\n')

artists:   0%|          | 0/15 [00:00<?, ?it/s]

Querying for Slayer - Reign in Blood


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Kept 44 out of 50 matches for Slayer - Reign in Blood

Querying for Metallica - Kill 'Em All


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

...

Kept 17 out of 50 matches for Burzum - Filosofem

Querying for Death - Symbolic


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Kept 24 out of 50 matches for Death - Symbolic

Querying for Dissection - Storm of the Light's Bane


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Kept 49 out of 50 matches for Dissection - Storm of the Light's Bane



In [41]:
results_df

,album,album_instance,artist,lastfm_info
0,Reign in Blood,Reign in Blood,Slayer,"{'name': 'Reign in Blood', 'artist': 'Slayer',..."
1,Reign in Blood,Reign In Blood (Expanded),Slayer,"{'name': 'Reign In Blood (Expanded)', 'artist'..."
2,Reign in Blood,Reign In Blood (Expanded Edition),Slayer,"{'name': 'Reign In Blood (Expanded Edition)', ..."
3,Reign in Blood,Reign In Blood (1994 Reissue),Slayer,"{'name': 'Reign In Blood (1994 Reissue)', 'art..."
4,Reign in Blood,Reign In Blood (Remastered),Slayer,"{'name': 'Reign In Blood (Remastered)', 'artis..."
...,...,...,...,...
490,Storm of the Light's Bane,Storm Of The Light's Bane [Disc 01],Dissection,{'name': 'Storm Of The Light's Bane [Disc 01]'...
491,Storm of the Light's Bane,Storm Of The Light's Bane Bonus,Dissection,"{'name': 'Storm Of The Light's Bane Bonus', 'a..."
492,Storm of the Light's Bane,Storm Of The Light's Bane \\\\ Where Dead Ange...,Dissection,{'name': 'Storm Of The Light's Bane \\\\ Where...
493,Storm of the Light's Bane,Storm of the Light's Bane Ultimate Reissue CD 2,Dissection,{'name': 'Storm of the Light's Bane Ultimate R...


Add some entries of the lastfm info as column to the dataframe.

In [42]:
df = results_df
df['listeners'] = df.progress_apply(lambda row: int(row['lastfm_info']['listeners']), axis=1)
df['playcount'] = df.progress_apply(lambda row: int(row['lastfm_info']['playcount']), axis=1)
df

100%|██████████| 495/495 [00:00<00:00, 140901.29it/s]


,album,album_instance,artist,lastfm_info,listeners,playcount
0,Reign in Blood,Reign in Blood,Slayer,"{'name': 'Reign in Blood', 'artist': 'Slayer',...",822151,15579798
1,Reign in Blood,Reign In Blood (Expanded),Slayer,"{'name': 'Reign In Blood (Expanded)', 'artist'...",165687,2127792
2,Reign in Blood,Reign In Blood (Expanded Edition),Slayer,"{'name': 'Reign In Blood (Expanded Edition)', ...",12823,372560
3,Reign in Blood,Reign In Blood (1994 Reissue),Slayer,"{'name': 'Reign In Blood (1994 Reissue)', 'art...",4856,130026
4,Reign in Blood,Reign In Blood (Remastered),Slayer,"{'name': 'Reign In Blood (Remastered)', 'artis...",2092,75516
...,...,...,...,...,...,...
490,Storm of the Light's Bane,Storm Of The Light's Bane [Disc 01],Dissection,{'name': 'Storm Of The Light's Bane [Disc 01]'...,1,573
491,Storm of the Light's Bane,Storm Of The Light's Bane Bonus,Dissection,"{'name': 'Storm Of The Light's Bane Bonus', 'a...",4,527
492,Storm of the Light's Bane,Storm Of The Light's Bane \\\\ Where Dead Ange...,Dissection,{'name': 'Storm Of The Light's Bane \\\\ Where...,10,265
493,Storm of the Light's Bane,Storm of the Light's Bane Ultimate Reissue CD 2,Dissection,{'name': 'Storm of the Light's Bane Ultimate R...,1,266


Sum up the different entries that correspond to the same album:

In [43]:
#TODO: Should probably be part of the utils.py
cumulative_df = df.drop(['album_instance'], axis=1).groupby(['artist','album']).sum()
cumulative_df

listeners  playcount
artist        album                                          
Agalloch      The Mantle                    352832   12132834
Black Sabbath Paranoid                     1937226   21612793
Burzum        Filosofem                     594290   13026055
              Hvis lyset tar oss            224814    3189025
Death         Symbolic                      251175    6732890
Dissection    Storm of the Light's Bane     378476    9735078
Hades Archer  Penis Metal                      489       9519
Iron Maiden   Iron Maiden                  8061592  137694269
              The Number of the Beast      1071172   15060267
Megadeth      Rust in Peace                 767270   16163950
Metallica     Kill 'Em All                  673071   16147392
              Master of Puppets            1231824   25628896
              Ride the Lightning           1163720   23614081
Morbid Angel  Altars of Madness             460416    9643752
Slayer        Reign in Blood               1027288   18550929

## Get tags of the most relevant album match

In [46]:
#TODO: Move these functions to the data_query_functions.py !
def get_tags(x):
    tag_names = []
    for tag in x['tags']['tag']:
        tag_names.append(tag['name'])
    return tag_names

def get_url(x):
    return x['url']

def get_info(x):
    artist = x.name[0]
    album = x.name[1]
    lastfm = dqf.LastFM()
    album_info = lastfm.get_album_info(artist, album)
    return album_info

def get_img(x):
    return x['image']

# TODO: Release date not present. Should use another API for that?
# def get_releasedate(x):
#     return x['releasedate']

def get_mbid(x):
    try:
        return x['mbid']
    except KeyError:
        return None

import requests
import pandas as pd
import xmltodict

def get_release_date(x):
    print(x)
    if x is not None:
        response = requests.get('http://musicbrainz.org/ws/2/release/' + str(x) + '?inc=release-groups&fmt=xml')
        response_dict = xmltodict.parse(response.text)
        return response_dict['metadata']['release']['release-group']['first-release-date']
    else:
        return None




In [47]:
cumulative_df['lastfm_info'] = cumulative_df.apply(lambda row: get_info(row), axis=1)
cumulative_df['tags'] = cumulative_df.apply(lambda row: get_tags(row['lastfm_info']), axis=1)
cumulative_df['url'] = cumulative_df.apply(lambda row: get_url(row['lastfm_info']), axis=1)
cumulative_df['images'] = cumulative_df.apply(lambda row: get_img(row['lastfm_info']), axis=1)
cumulative_df['mbid'] = cumulative_df.apply(lambda row: get_mbid(row['lastfm_info']), axis=1)
cumulative_df['release_date'] = cumulative_df.apply(lambda row: get_release_date(row['mbid']), axis=1)
cumulative_df.drop(['lastfm_info', 'url', 'images'], axis=1)


7ac99528-77a9-3624-84b7-3400f6f56e47
2982b682-36ea-3605-b959-04e746736070
1ce9177c-62a0-4403-a7ee-7359026fcbf6
b6928219-0772-39ac-8156-91a609b2fd5e
321a3c33-9310-4b9f-b104-762e465ec60f
76a7d348-f6e2-4e36-b529-b3cc003c3231
None
25da813d-4dbd-32c0-aef0-307e790f0709
6ea83f20-d053-3495-bc25-76aa61da13ab
2b904e74-daba-397c-a151-bafb125ceb44
c06ed440-f25d-3127-aadb-ebe9c685b3d8
b30c90dd-83f1-4897-ae36-58fb3935e5ee
2236dd07-a2f3-466a-973d-9069001a89da
0e47ca20-9a60-3bd0-afe0-a1e090a75f5e
e80dcb10-f532-4da1-9458-afd541e4a9ea


listeners  playcount  \
artist        album                                             
Agalloch      The Mantle                    352832   12132834   
Black Sabbath Paranoid                     1937226   21612793   
Burzum        Filosofem                     594290   13026055   
              Hvis lyset tar oss            224814    3189025   
Death         Symbolic                      251175    6732890   
Dissection    Storm of the Light's Bane     378476    9735078   
Hades Archer  Penis Metal                      489       9519   
Iron Maiden   Iron Maiden                  8061592  137694269   
              The Number of the Beast      1071172   15060267   
Megadeth      Rust in Peace                 767270   16163950   
Metallica     Kill 'Em All                  673071   16147392   
              Master of Puppets            1231824   25628896   
              Ride the Lightning           1163720   23614081   
Morbid Angel  Altars of Madness             460416    9643752   
Slayer        Reign in Blood               1027288   18550929   

                                                                                      tags  \
artist        album                                                                          
Agalloch      The Mantle                 [folk metal, doom metal, black metal, albums I...   
Black Sabbath Paranoid                   [heavy metal, hard rock, albums I own, classic...   
Burzum        Filosofem                  [black metal, ambient black metal, atmospheric...   
              Hvis lyset tar oss         [black metal, ambient black metal, atmospheric...   
Death         Symbolic                   [death metal, progressive death metal, Technic...   
Dissection    Storm of the Light's Bane  [black metal, melodic black metal, albums I ow...   
Hades Archer  Penis Metal                                                               []   
Iron Maiden   Iron Maiden                 [heavy metal, albums I own, NWOBHM, metal, 1980]   
              The Number of the Beast    [heavy metal, albums I own, NWOBHM, metal, Iro...   
Megadeth      Rust in Peace              [thrash metal, albums I own, speed metal, heav...   
Metallica     Kill 'Em All               [thrash metal, albums I own, metal, heavy meta...   
              Master of Puppets          [thrash metal, albums I own, metal, heavy meta...   
              Ride the Lightning         [thrash metal, albums I own, metal, heavy meta...   
Morbid Angel  Altars of Madness          [death metal, 1989, albums I own, old school d...   
Slayer        Reign in Blood             [thrash metal, albums I own, metal, speed meta...   

                                                                         mbid  \
artist        album                                                             
Agalloch      The Mantle                 7ac99528-77a9-3624-84b7-3400f6f56e47   
Black Sabbath Paranoid                   2982b682-36ea-3605-b959-04e746736070   
Burzum        Filosofem                  1ce9177c-62a0-4403-a7ee-7359026fcbf6   
              Hvis lyset tar oss         b6928219-0772-39ac-8156-91a609b2fd5e   
Death         Symbolic                   321a3c33-9310-4b9f-b104-762e465ec60f   
Dissection    Storm of the Light's Bane  76a7d348-f6e2-4e36-b529-b3cc003c3231   
Hades Archer  Penis Metal                                                None   
Iron Maiden   Iron Maiden                25da813d-4dbd-32c0-aef0-307e790f0709   
              The Number of the Beast    6ea83f20-d053-3495-bc25-76aa61da13ab   
Megadeth      Rust in Peace              2b904e74-daba-397c-a151-bafb125ceb44   
Metallica     Kill 'Em All               c06ed440-f25d-3127-aadb-ebe9c685b3d8   
              Master of Puppets          b30c90dd-83f1-4897-ae36-58fb3935e5ee   
              Ride the Lightning         2236dd07-a2f3-466a-973d-9069001a89da   
Morbid Angel  Altars of Madness          0e47ca20-9a60-3bd0-afe0-a1e090a75f5e   
Slayer        Reign in Blood            

In [30]:
cumulative_df.drop('lastfm_info', axis=1).sort_values(by='listeners', ascending=False).to_csv('./data/proc_MA_10k_albums.csv')